In [40]:
import pandas as pd
from sklearn.impute import KNNImputer
import joblib

In [41]:
dict_particulas = joblib.load('estaciones.joblib')
dict_particulas.keys()

dict_keys(['pm_2.5_inia', 'pm_2.5_puren', 'pm_10_inia', 'pm_10_puren'])

In [42]:
#PARA PM2.5
# RECORRER DIA A DIA EN AMBAS ESTACIONES
# TRATAR SIGUIENTES CASOS: 
# si existen valores en una estacion solamente entonces dejar el valor existente
# si existen valores en ambas estaciones entonces promediar


df_puren25 = dict_particulas['pm_2.5_puren']
df_inia25 = dict_particulas['pm_2.5_inia']

# Establecer la columna de fecha como índice
df_puren25.reset_index(inplace=True)
df_inia25.reset_index(inplace=True)

df_puren25.set_index('fecha', inplace=True)
df_inia25.set_index('fecha', inplace=True)

# Combinar los dos data frames
df_combinado25 = pd.concat([df_puren25, df_inia25], keys=['puren', 'inia'], axis=1)


#PARA PM10
# RECORRER DIA A DIA EN AMBAS ESTACIONES
# TRATAR SIGUIENTES CASOS: 
# si existen valores en una estacion solamente entonces dejar el valor existente
# si existen valores en ambas estaciones entonces promediar

df_puren10 = dict_particulas['pm_10_puren']
df_inia10 = dict_particulas['pm_10_inia']

# Establecer la columna de fecha como índice
df_puren10.reset_index(inplace=True)
df_inia10.reset_index(inplace=True)

df_puren10.set_index('fecha', inplace=True)
df_inia10.set_index('fecha', inplace=True)

# Combinar los dos data frames
df_combinado10 = pd.concat([df_puren10, df_inia10], keys=['puren', 'inia'], axis=1)




In [43]:
# Lidiar con NaNs y calcular el promedio donde sea necesario
def promedio_o_existente(row):
    valor_puren = row[('puren', 'registro')]
    valor_inia = row[('inia', 'registro')]
    
    if pd.isna(valor_puren) and pd.isna(valor_inia):
        return np.nan
    elif pd.isna(valor_puren):
        return valor_inia
    elif pd.isna(valor_inia):
        return valor_puren
    else:
        return (valor_puren + valor_inia) / 2



# Aplicar la función a todas las filas
df_combinado25['promedio_o_existente'] = df_combinado25.apply(promedio_o_existente, axis=1)

# Mostrar el resultado
print(df_combinado25)


             puren               inia          promedio_o_existente
             index   registro   index registro                     
fecha                                                              
2012-12-31     0.0   7.318980     NaN      NaN             7.318980
2013-01-01     1.0   9.361155     NaN      NaN             9.361155
2013-01-02     2.0  13.786540     NaN      NaN            13.786540
2013-01-03     3.0  12.311413     0.0  12.2021            12.256756
2013-01-04     4.0  12.694497     NaN      NaN            12.694497
...            ...        ...     ...      ...                  ...
2023-03-27  3711.0  20.166667  3222.0  17.0000            18.583333
2023-03-28  3712.0   6.625000  3223.0   5.0000             5.812500
2023-03-29  3713.0   5.708333  3224.0   5.0000             5.354167
2023-03-30  3714.0  11.666667  3225.0   9.0000            10.333333
2023-03-31  3715.0  22.666667  3226.0  19.0000            20.833333

[3734 rows x 5 columns]


In [44]:
df_combinado10['promedio_o_existente'] = df_combinado10.apply(promedio_o_existente, axis=1)
# Mostrar el resultado
print(df_combinado10.head())

           puren           inia          promedio_o_existente
           index registro index registro                     
fecha                                                        
2013-01-01   0.0  27.4733   0.0  23.0059             25.23960
2013-01-02   1.0  40.6500   1.0  31.5644             36.10720
2013-01-03   2.0  32.1830   2.0  26.2818             29.23240
2013-01-04   3.0  38.0133   3.0  28.3704             33.19185
2013-01-05   4.0  30.8071   4.0  21.8812             26.34415


In [45]:
df_pm25 = df_combinado25.drop(['puren', 'inia'], axis=1, level=0)

df_pm25.reset_index(inplace=True)

#asignar los nombres de las columnas
df_pm25.columns = ['fecha', 'registro']

print('data pm2.5 \n\n',df_pm25.head())



data pm2.5 

        fecha   registro
0 2012-12-31   7.318980
1 2013-01-01   9.361155
2 2013-01-02  13.786540
3 2013-01-03  12.256756
4 2013-01-04  12.694497


In [46]:
df_pm10 = df_combinado10.drop(['puren', 'inia'], axis=1, level=0)

df_pm10.reset_index(inplace=True)

#asignar los nombres de las columnas
df_pm10.columns = ['fecha', 'registro']

print('data pm10 \n\n',df_pm10.head())

data pm10 

        fecha  registro
0 2013-01-01  25.23960
1 2013-01-02  36.10720
2 2013-01-03  29.23240
3 2013-01-04  33.19185
4 2013-01-05  26.34415


In [47]:
# Establecer la columna de fecha como índice
df_pm25.set_index('fecha', inplace=True)
df_pm10.set_index('fecha', inplace=True)

# Rellenar fechas faltantes
df_pm25 = df_pm25.resample('D').asfreq()
df_pm10 = df_pm10.resample('D').asfreq()


# Contar el número de NaN en cada columna del DataFrame df_filled
nan_count_df_pm25 = df_pm25.isna().sum()
nan_count_df_pm10 = df_pm10.isna().sum()


#mostrar los valores nulos existentes en cada data frame 
print('pm25 nan: ',nan_count_df_pm25)
print('pm10 nan: ',nan_count_df_pm10)

df_pm10 = df_pm10.reset_index()
df_pm25 = df_pm25.reset_index()


pm25 nan:  registro    9
dtype: int64
pm10 nan:  registro    11
dtype: int64


In [48]:
rows_with_nan = df_pm25[df_pm25.isna().any(axis=1)]
print('missing data pm25 \n\n',rows_with_nan)

rows_with_nan1 = df_pm10[df_pm10.isna().any(axis=1)]
print('missing data pm10 \n\n',rows_with_nan1)

missing data pm25 

           fecha  registro
47   2013-02-16       NaN
48   2013-02-17       NaN
49   2013-02-18       NaN
50   2013-02-19       NaN
118  2013-04-28       NaN
119  2013-04-29       NaN
120  2013-04-30       NaN
1443 2016-12-13       NaN
1444 2016-12-14       NaN
missing data pm10 

           fecha  registro
613  2014-09-06       NaN
614  2014-09-07       NaN
784  2015-02-24       NaN
785  2015-02-25       NaN
786  2015-02-26       NaN
787  2015-02-27       NaN
969  2015-08-28       NaN
1236 2016-05-21       NaN
1237 2016-05-22       NaN
1371 2016-10-03       NaN
3437 2022-05-31       NaN


In [49]:
# Convertir las fechas a formato numérico (por ejemplo, número de días desde la fecha mínima)
df_pm25['fecha_num'] = (pd.to_datetime(df_pm25['fecha']) - pd.to_datetime(df_pm25['fecha']).min()).dt.days


# Preparar los datos para la imputación
X = df_pm25[['fecha_num', 'registro']].values

# Realizar la imputación usando k-NN

knn_imputer = KNNImputer(n_neighbors=2)
X_imputed = knn_imputer.fit_transform(X)

# Crear un nuevo DataFrame con los valores imputados
df_imputed_pm25 = pd.DataFrame(X_imputed, columns=['fecha', 'registro'])

# Reemplazar la columna antigua por la imputada
df_pm25['registro'] = df_imputed_pm25['registro']

df_pm25 = df_pm25.drop('fecha_num', axis=1)



In [50]:
# Convertir las fechas a formato numérico (por ejemplo, número de días desde la fecha mínima)
df_pm10['fecha_num'] = (pd.to_datetime(df_pm10['fecha']) - pd.to_datetime(df_pm10['fecha']).min()).dt.days


# Preparar los datos para la imputación
X = df_pm10[['fecha_num', 'registro']].values

# Realizar la imputación usando k-NN
knn_imputer = KNNImputer(n_neighbors=2)
X_imputed = knn_imputer.fit_transform(X)

# Crear un nuevo DataFrame con los valores imputados
df_imputed_pm10 = pd.DataFrame(X_imputed, columns=['fecha', 'registro'])


# reemplazar la columna antigua por la imputada
df_pm10['registro'] = df_imputed_pm10['registro']

df_pm10 = df_pm10.drop('fecha_num', axis=1)


In [51]:

rows_with_nan = df_pm25[df_pm25.isna().any(axis=1)]
print('missing data pm25 \n\n',rows_with_nan)

rows_with_nan1 = df_imputed_pm10[df_imputed_pm10.isna().any(axis=1)]
print('\n\n missing data pm10 \n\n',rows_with_nan1)

missing data pm25 

 Empty DataFrame
Columns: [fecha, registro]
Index: []


 missing data pm10 

 Empty DataFrame
Columns: [fecha, registro]
Index: []


In [52]:
dict_combined ={
    'pm_2.5': df_pm25,
    'pm_10': df_pm10
}


for clave, df in dict_combined.items():
    # path para el guardado de archivos
    path = '../../data/processed/combined/{}.csv'.format(clave)
    df.to_csv(path, sep=';',index=False, float_format='%.4f', decimal=',')

#exportamos diccionario
joblib.dump(dict_combined, 'contaminantes.joblib')

['contaminantes.joblib']